In [31]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from dataset import load_obj, extracted_data, extracted_data_name
from sklearn.metrics import  f1_score

In [32]:
X, labels = load_obj(os.path.join(extracted_data, extracted_data_name))

In [33]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# кодируем таргет
le = LabelEncoder()
y = le.fit_transform(labels)
ohe =  OneHotEncoder()
y_ohe = ohe.fit_transform(y.reshape(-1,1))

In [34]:
n_features = X.shape[1]
n_classes = y.max()+1
print('Число классов: {}\nЧисло признаков: {}\nЧисло сэмплов: {}'.format(n_classes, n_features, y.shape[0]))

Число классов: 8
Число признаков: 193
Число сэмплов: 11307


In [35]:
# сплитим
X_train, X_test, y_train, y_test, y_ohe_train, y_ohe_test = train_test_split(X, y, y_ohe, \
                                                                             train_size=0.8, test_size=0.2)

C_grid = np.logspace(-3,4,10) #[1e-3,1e-2,1e-1,1e1,1e2]
lr = LogisticRegressionCV(C_grid, n_jobs=-1) #LogisticRegression()

In [36]:
# # Обучаем
# print('Training...')
# lr.fit(X_train, y_train)

In [37]:
# # Оцениваем качества на отложенной выборке
# print('Scoring...')
# tr_score = lr.score(X_train, y_train)
# te_score = lr.score(X_test, y_test)
# print('Acc:: Train score: {}, test score: {}'.format(tr_score, te_score))

# tr_score = f1_score(y_train, lr.predict(X_train), average='weighted')
# te_score = f1_score(y_test, lr.predict(X_test), average='weighted')
# print('F1:: Train score: {}, test score: {}'.format(tr_score, te_score))


## Нелинейные модели

#### онлайн-обучение

In [89]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import backend as K
from keras.optimizers import Adam

In [93]:
K.clear_session()
model = Sequential()
model.add(Dense(100, input_dim=n_features))
model.add(Dense(1200, activation='relu'))
model.add(Dense(1200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Activation('relu'))
model.add(Dense(n_classes, activation='softmax'))

In [94]:
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(1e-5), 
              metrics=['accuracy'])

In [95]:
model.fit(X_train, y_ohe_train.todense(), epochs=300, batch_size=250, validation_split=0.2)

Train on 7236 samples, validate on 1809 samples
Epoch 1/300
7236/7236 [==============================] - 0s 60us/step - loss: 8.3318 - acc: 0.2204 - val_loss: 4.1699 - val_acc: 0.4113
Epoch 2/300
7236/7236 [==============================] - 0s 42us/step - loss: 6.5475 - acc: 0.3147 - val_loss: 3.3461 - val_acc: 0.5368
Epoch 3/300
7236/7236 [==============================] - 0s 43us/step - loss: 5.6316 - acc: 0.3726 - val_loss: 2.4364 - val_acc: 0.5318
Epoch 4/300
7236/7236 [==============================] - 0s 41us/step - loss: 4.7720 - acc: 0.3995 - val_loss: 1.8753 - val_acc: 0.5749
Epoch 5/300
7236/7236 [==============================] - 0s 41us/step - loss: 4.0767 - acc: 0.4175 - val_loss: 1.5549 - val_acc: 0.5456
Epoch 6/300
7236/7236 [==============================] - 0s 41us/step - loss: 3.3870 - acc: 0.4316 - val_loss: 1.3636 - val_acc: 0.5898
Epoch 7/300
7236/7236 [==============================] - 0s 41us/step - loss: 2.8619 - acc: 0.4402 - val_loss: 1.2926 - val_acc: 0.5638


7236/7236 [==============================] - 0s 41us/step - loss: 0.3815 - acc: 0.8745 - val_loss: 0.2743 - val_acc: 0.9237
Epoch 61/300
7236/7236 [==============================] - 0s 40us/step - loss: 0.3864 - acc: 0.8706 - val_loss: 0.2795 - val_acc: 0.9210
Epoch 62/300
7236/7236 [==============================] - 0s 40us/step - loss: 0.3785 - acc: 0.8776 - val_loss: 0.2756 - val_acc: 0.9160
Epoch 63/300
7236/7236 [==============================] - 0s 40us/step - loss: 0.3675 - acc: 0.8813 - val_loss: 0.2639 - val_acc: 0.9287
Epoch 64/300
7236/7236 [==============================] - 0s 44us/step - loss: 0.3748 - acc: 0.8774 - val_loss: 0.2690 - val_acc: 0.9182
Epoch 65/300
7236/7236 [==============================] - 0s 45us/step - loss: 0.3671 - acc: 0.8784 - val_loss: 0.2608 - val_acc: 0.9287
Epoch 66/300
7236/7236 [==============================] - 0s 44us/step - loss: 0.3524 - acc: 0.8857 - val_loss: 0.2580 - val_acc: 0.9281
Epoch 67/300
7236/7236 [==============================

Epoch 120/300
7236/7236 [==============================] - 0s 42us/step - loss: 0.2288 - acc: 0.9255 - val_loss: 0.1647 - val_acc: 0.9547
Epoch 121/300
7236/7236 [==============================] - 0s 40us/step - loss: 0.2241 - acc: 0.9346 - val_loss: 0.1651 - val_acc: 0.9519
Epoch 122/300
7236/7236 [==============================] - 0s 41us/step - loss: 0.2182 - acc: 0.9310 - val_loss: 0.1615 - val_acc: 0.9563
Epoch 123/300
7236/7236 [==============================] - 0s 42us/step - loss: 0.2201 - acc: 0.9299 - val_loss: 0.1622 - val_acc: 0.9547
Epoch 124/300
7236/7236 [==============================] - 0s 41us/step - loss: 0.2204 - acc: 0.9297 - val_loss: 0.1597 - val_acc: 0.9541
Epoch 125/300
7236/7236 [==============================] - 0s 40us/step - loss: 0.2189 - acc: 0.9333 - val_loss: 0.1588 - val_acc: 0.9563
Epoch 126/300
7236/7236 [==============================] - 0s 40us/step - loss: 0.2213 - acc: 0.9345 - val_loss: 0.1552 - val_acc: 0.9591
Epoch 127/300
7236/7236 [=========

7236/7236 [==============================] - 0s 40us/step - loss: 0.1518 - acc: 0.9538 - val_loss: 0.1137 - val_acc: 0.9701
Epoch 180/300
7236/7236 [==============================] - 0s 45us/step - loss: 0.1494 - acc: 0.9570 - val_loss: 0.1157 - val_acc: 0.9657
Epoch 181/300
7236/7236 [==============================] - 0s 43us/step - loss: 0.1523 - acc: 0.9522 - val_loss: 0.1105 - val_acc: 0.9718
Epoch 182/300
7236/7236 [==============================] - 0s 42us/step - loss: 0.1510 - acc: 0.9544 - val_loss: 0.1123 - val_acc: 0.9668
Epoch 183/300
7236/7236 [==============================] - 0s 40us/step - loss: 0.1523 - acc: 0.9534 - val_loss: 0.1100 - val_acc: 0.9679
Epoch 184/300
7236/7236 [==============================] - 0s 41us/step - loss: 0.1484 - acc: 0.9540 - val_loss: 0.1111 - val_acc: 0.9701
Epoch 185/300
7236/7236 [==============================] - 0s 45us/step - loss: 0.1460 - acc: 0.9591 - val_loss: 0.1060 - val_acc: 0.9701
Epoch 186/300
7236/7236 [=======================

7236/7236 [==============================] - 0s 45us/step - loss: 0.1048 - acc: 0.9684 - val_loss: 0.0866 - val_acc: 0.9751
Epoch 239/300
7236/7236 [==============================] - 0s 44us/step - loss: 0.1014 - acc: 0.9711 - val_loss: 0.0803 - val_acc: 0.9757
Epoch 240/300
7236/7236 [==============================] - 0s 40us/step - loss: 0.1041 - acc: 0.9688 - val_loss: 0.0818 - val_acc: 0.9768
Epoch 241/300
7236/7236 [==============================] - 0s 42us/step - loss: 0.0994 - acc: 0.9695 - val_loss: 0.0789 - val_acc: 0.9779
Epoch 242/300
7236/7236 [==============================] - 0s 41us/step - loss: 0.1033 - acc: 0.9710 - val_loss: 0.0813 - val_acc: 0.9757
Epoch 243/300
7236/7236 [==============================] - 0s 42us/step - loss: 0.0983 - acc: 0.9695 - val_loss: 0.0809 - val_acc: 0.9757
Epoch 244/300
7236/7236 [==============================] - 0s 41us/step - loss: 0.0960 - acc: 0.9711 - val_loss: 0.0782 - val_acc: 0.9768
Epoch 245/300
7236/7236 [=======================

7236/7236 [==============================] - 0s 41us/step - loss: 0.0723 - acc: 0.9804 - val_loss: 0.0640 - val_acc: 0.9812
Epoch 298/300
7236/7236 [==============================] - 0s 40us/step - loss: 0.0739 - acc: 0.9778 - val_loss: 0.0612 - val_acc: 0.9823
Epoch 299/300
7236/7236 [==============================] - 0s 41us/step - loss: 0.0685 - acc: 0.9798 - val_loss: 0.0610 - val_acc: 0.9834
Epoch 300/300
7236/7236 [==============================] - 0s 41us/step - loss: 0.0731 - acc: 0.9779 - val_loss: 0.0623 - val_acc: 0.9818


In [97]:
def inverse_ohe(ohe_outputs, ohe_encoder):
    return ohe_encoder.active_features_[ohe_outputs.argmax(axis=1)]

In [98]:
preds = inverse_ohe(model.predict(X_train),ohe)
tr_score = f1_score(y_train, preds, average='weighted')
preds = inverse_ohe(model.predict(X_test),ohe)
te_score = f1_score(y_test, preds, average='weighted')
print('F1:: Train score: {}, test score: {}'.format(tr_score, te_score))


F1:: Train score: 0.9886144235151906, test score: 0.9809463477365804
